**Computes hourly departures and arrivals per h3 cells (level 9)**

**Input**: csv using the following columns:
* `transportation_mode`: used mode of transport for the trip (see table below)
* `start_time`: datetime of begining of trip
* `end_time`: datetime of end of trip
* `user_id`: Id of the traveling user, used to make sure results include more than one user per geographic division
* `trace_gps`: list of (lon, lat) tuples

**Output**: geoJSON files for origins (departures) and destination (arrivals) for each hour (eg. "../static/data/origins/origins_1h.geojson") containing h3 cell shapes with the following metadata:
* `trace_gps`: (Badly named) number of trips starting or ending in cell
* `color` and `fill`: A color representation of the number of trips, from green to red

For anonymity, there must be at least 3 trips in a given cell for data to be considered


The notebook also displays the first few GPS points starting from a given cell, color coded by transport mode.
This feature helps explore mobility behaviour around cells containing a landmark for olympic games.
For now, it isn't exported to frontend.

In [ ]:
import pandas as pd
import geopandas as gpd
import h3pandas
from shapely.geometry import Point, Polygon, LineString
import json
import folium

In [ ]:
df_src = pd.read_csv("sources/data.csv")

In [ ]:
tr = {
-10 : "NOT_DEFINED",
0 : "UNKNOWN",
1 : "PASSENGER_CAR",
2 : "MOTORCYCLE",
3 : "HEAVY_DUTY_VEHICLE",
4 : "BUS",
5 : "COACH",
6 : "RAIL_TRIP",
7 : "BOAT_TRIP",
8 : "BIKE_TRIP",
9 : "PLANE",
10 : "SKI",
11 : "FOOT",
12 : "IDLE",
13 : "OTHER",
101 : "SCOOTER",
102 : "HIGH_SPEED_TRAIN"
}
df_src['transportation_mode_tr'] = df_src['transportation_mode'].apply(lambda x: tr[x])

In [ ]:
#df = df_src[df_src['transportation_mode_tr'] == "FOOT"]
df = df_src

In [ ]:
#dfh3 = df.h3.geo_to_h3_aggregate(9, operation="count", lat_col="latitude", lng_col="longitude")
dfh3 = df.h3.geo_to_h3(9, lat_col="latitude", lng_col="longitude", set_index=False).rename(columns={"h3_09": "h3_"})
times = pd.DatetimeIndex(dfh3.start_time)
dfh3['start_time'] = pd.to_datetime(dfh3['start_time'])
dfh3['start_hour'] = dfh3['start_time'].dt.hour
drawgeoframe = dfh3.groupby(['h3_', 'start_hour']).agg('count').reset_index()[['h3_', 'start_hour','trace_gps']].set_index('h3_')

drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()
drawgeoframe

In [ ]:
import folium
import branca.colormap as cm
colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=0, vmax=drawgeoframe["trace_gps"].max())

drawgeoframe["color"] = drawgeoframe["trace_gps"].apply(lambda x: colormap(x)[:-2])
drawgeoframe["fill"] = drawgeoframe["color"]
drawgeoframe = drawgeoframe[drawgeoframe["trace_gps"]>2]
start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
# other mapping code (e.g. lines, markers etc.)
for i in range(0,24):
    folium.GeoJson(drawgeoframe[drawgeoframe['start_hour']==i], name=i, style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
    drawgeoframe[drawgeoframe['start_hour']==i].to_file("../static/data/origins/origins_" + str(i) + "h.geojson", driver="GeoJSON")
folium.LayerControl().add_to(m)
m

In [ ]:
import json
def extract_end_coords(t):
    return json.loads(t)[-1]
df['lon_end'], df['lat_end'] = zip(*df["trace_gps"].map(extract_end_coords))
dfh3 = df.h3.geo_to_h3(9, lat_col="lat_end", lng_col="lon_end", set_index=False).rename(columns={"h3_09": "h3_"})
times = pd.DatetimeIndex(dfh3.end_time)
dfh3['end_time'] = pd.to_datetime(dfh3['end_time'])
dfh3['end_hour'] = dfh3['end_time'].dt.hour
drawgeoframe = dfh3.groupby(['h3_', 'end_hour']).agg('count').reset_index()[['h3_', 'end_hour','trace_gps']].set_index('h3_')

drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()
drawgeoframe["color"] = drawgeoframe["trace_gps"].apply(lambda x: colormap(x)[:-2])
drawgeoframe["fill"] = drawgeoframe["color"]
drawgeoframe = drawgeoframe[drawgeoframe["trace_gps"]>2]
start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
# other mapping code (e.g. lines, markers etc.)
for i in range(0,24):
    folium.GeoJson(drawgeoframe[drawgeoframe['end_hour']==i], name=i, style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
    drawgeoframe[drawgeoframe['end_hour']==i].to_file("../static/data/destinations/destinations_" + str(i) + "h.geojson", driver="GeoJSON")
folium.LayerControl().add_to(m)
m

In [ ]:
h3_saint_lazare = ["891fb475b37ffff"]
h3_gare_du_nord = ["891fb4660dbffff"]
h3_gare_de_lest = ["891fb466053ffff", "891fb4660cfffff", "891fb4660cbffff"]
h3_addidas_arena = ["891fb4664afffff", "891fb4664abffff", "891fb466433ffff"]
h3_pont_alexandre_trois = ["891fb4675d3ffff"]
h3_invalides = ["891fb4675dbffff"]
h3_grand_palais = ["891fb475b6bffff"]
h3_tour_eiffel = ["891fb46741bffff", "891fb467413ffff", "891fb467403ffff", "891fb46740bffff"]
h3_trocadero = ["891fb4674d7ffff"]
h3_grand_palais_ephemere = ["891fb467477ffff"]
h3_place_concorde = ["891fb46759bffff"]
h3_porte_versailles = ["891fb467673ffff"]
h3_arena_bercy = ["891fb46440fffff"]
h3_parc_des_princes = ["891fb462b8bffff","891fb462b8fffff", "891fb462b13ffff"]
h3_rolland_garros = ["891fb462867ffff"]
h3_la_defense_arena = ["891fb475313ffff", "891fb47538fffff"]
h3_stade_de_france = ["891fb474b83ffff", "891fb474b9bffff", "891fb474b93ffff", "891fb474b97ffff", "891fb474b87ffff"]
dfh3_filter = dfh3[dfh3["h3_"].isin(h3_saint_lazare)]
def extract_coords(t):
    return LineString(json.loads(t)[0:2])
dfh3_filter['trace_geom']= dfh3_filter["trace_gps"].apply(extract_coords)

In [ ]:
geo_df = gpd.GeoDataFrame(dfh3_filter,geometry='trace_geom',crs='epsg:4326')

In [ ]:
def transportation_mode_tr_colors(x):
    if x == "RAIL_TRIP":
        return "red"
    if x == "HIGH_SPEED_TRAIN":
        return "orange"
    if x == "FOOT":
        return "lightgreen"
    if x == "BIKE_TRIP":
        return "green"
    if x == "PASSENGER_CAR":
        return "black"
    return "blue"
geo_df["color"] = geo_df["transportation_mode_tr"].apply(lambda x: transportation_mode_tr_colors(x))
m = folium.Map(location=[48.8915079, 2.3495425], zoom_start=13)
folium.GeoJson(geo_df[['trace_geom', 'color']], style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
m